<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 4: Introduction to Simulation with Python (Part 6)

Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)


<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Logging (Advanced: *EventLogger*)
* Containers


Credits: The following content is adapted from the official [gas station refueling example](https://simpy.readthedocs.io/en/3.0/examples/gas_station_refuel.html) 

# Gas Station Refueling Example

### Scenario

- This examples models a gas station and cars that arrive at the station for refueling.

- The gas station has a limited number of fuel pumps and a fuel tank that is shared between the fuel pumps. 

- The gas station is thus modeled as Resource. The shared fuel tank is modeled with a Container.

- Vehicles arriving at the gas station first request a fuel pump from the station. Once they acquire one, they try to take the desired amount of fuel from the fuel pump. They leave when they are done.

- The gas stations fuel level is reqularly monitored by gas station control. When the level drops below a certain threshold, a tank truck is called to refuel the gas station itself.

In [ ]:
import simpy
import random
import pandas as pd


Let's recall our convention, 

- Resources will be modeled by a python class
- Entities will be modeled by a python class
- Entities arrivals will be modeled by a python function (*simpy process*)

Remember that when possible, we'll use this convention to model our system

### Event Logger

The `EventLogger` class is for logging all relevant events that occur within the simulation system. Note that:

- Every different event is a different method of the class (`log_`).
- Each event is a dictionary with relevant information of that particular event.
- Different events could have different relevant values to log. We model this through method parameters.
- We use a python list for store all the events. Every time a method (to log) is called, a new event dictionary is appended to the list.
- At the end, we can get the logs as a pandas DataFrame (`get_logs_df`) or we could dump it as a csv file (`dump_logs_df`)

With that, we have a DataFrame (table) we can manipulate to answer questions regarding the system.

In [3]:
class EventLogger:
    def __init__(self):
        self.logs = []
    
    def log_car_arrival(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_arrival', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})
    
    def log_car_fuel_request(self, entity, time, fuel_required, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_fuel_request', 'event_key': entity, 'fuel_required': fuel_required, 'fuel_tank_level': fuel_tank_level})

    def log_car_departure(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_departure', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})

    def log_truck_call(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_call', 'fuel_tank_level': fuel_tank_level})
    
    def log_truck_arrival(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_arrival', 'fuel_tank_level': fuel_tank_level})

    def log_truck_departure(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_departure', 'fuel_tank_level': fuel_tank_level})

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

### Resource (Gas Station)

We initialize the `GasStation` class with the simpy environment (`self.env`), the `EventLogger` object (instance) and two resources: fuel pump (`simpy.Resource`) and fuel tank (`simpy.Container`). We also store more parameters as attributes and we create a flag (boolean) attribute to check if the truck was (already) called.

The `refuel_car` method models the process that interacts within the `Car` flow and consumes (`.get()`) fuel from the fuel tank (which takes some time depending on the fuel and refueling speed)

Then, we have two other processes: 
1. `fuel_tank_control` will be in charge of monitoring the fuel tank periodically (every 10 seconds). Whenever the fuel tank level is below some defined threshold (and the truck was not already called) it will call the truck to refill the fuel tank (i.e. `yield` the `refill_fuel_tank` process)
2. `refill_fuel_tank` will be in charge of refilling the fuel tank:
    - First we change our flag attribute to `True` (why?) 
    - Then the truck takes some time to arrive the station which we model using the `tank_truck_time` attribute.
    - We compute the required amount to top up the fuel tank and refill it with the `.put()` method.
    - Finally, we set our flag attribute again to `False` (why?)

**Note how the logger is called at several points of the entity flow to log different events. We could delete the printing commands and still have all the information about what's happening in the system**

In [4]:
class GasStation:
    def __init__(self, env, n_fuel_pumps, station_tank_size, refueling_speed, tank_truck_time, logger):
        '''
        Initiaize the gas station with the following parameters:
        - env: the simpy environment
        - n_fuel_pumps: the number of fuel pumps at the station
        - station_tank_size: the size of the station's fuel tank
        - refueling_speed: the speed at which a car can refuel
        - tank_truck_time: the time it takes for the tank truck to arrive
        - logger: the logger object to log events
        '''

        self.env = env
        self.fuel_pump = simpy.Resource(self.env, capacity=n_fuel_pumps)
        self.fuel_tank = simpy.Container(self.env, capacity=station_tank_size, init=station_tank_size) #initialize the tank with the full capacity

        self.refueling_speed = refueling_speed
        self.tank_truck_time = tank_truck_time
        self.truck_called = False
        
        self.logger = logger


    def refuel_car(self, fuel_required):
        '''
        The refueling process for a car
        - fuel_required: the amount of fuel the car requires
        '''

        yield self.fuel_tank.get(fuel_required)
        # The "actual" refueling process takes some time
        yield self.env.timeout(fuel_required / self.refueling_speed)

    def refill_fuel_tank(self):
        '''
        The process for refilling the station's fuel tank
        '''
        self.truck_called = True
        
        yield self.env.timeout(self.tank_truck_time)
        self.logger.log_truck_arrival(self.env.now, self.fuel_tank.level)

        amount = self.fuel_tank.capacity - self.fuel_tank.level #top up the tank
        yield self.fuel_tank.put(amount)
        
        print(f'{self.env.now:6.1f} s: Tank truck arrived and refuelled station with {amount:.1f}L')
        self.logger.log_truck_departure(self.env.now, self.fuel_tank.level)
        self.truck_called = False

    def fuel_tank_control(self, threshold):
        '''
        The process that periodically checks the fuel tank level and calls the tank truck (refill_fuel_tank process) if the level falls below a threshold
        - threshold: the minimum fuel level that triggers the tank truck to be called
        '''

        while True:
            fuel_tank_level_ratio = 100 * (self.fuel_tank.level / self.fuel_tank.capacity)
            if (fuel_tank_level_ratio < threshold) and (not self.truck_called):
                # We need to call the tank truck now!
                print(f'{self.env.now:6.1f} s: Calling tank truck')
                self.logger.log_truck_call(self.env.now, self.fuel_tank.level)
                
                # Wait for the tank truck to arrive and refuel the station tank
                yield self.env.process(self.refill_fuel_tank())

            yield self.env.timeout(10) # Check every 10 seconds



### Entity (Car)

We initialize the `Car` class with the simpy environment (`self.env`), the `GasStation` and the `EventLogger` object (instance). 

We also store some attributes about the car such as the tank size (`self.car_tank_size`) and the current tank level (`self.car_tank_level`). We're assuming all cars have the same tank size but they arrive with different tank levels.

As with previous examples, we define the flow of our entity in the `run` method:

1. First, we request access to the simpy resource (`gas_station.fuel_pump.request()`). Recall that the simpy resource is stored in the `fuel_pump` attribute of the `gas_station` instance.
2. We compute the amount of fuel required to top up the car tank.
3. Finally, we call the `gas_station.refuel_car()` process with the specific amount of fuel required.

**Note how the logger is called at several points of the entity flow to log different events. We could delete the printing commands and still have all the information about what's happening in the system**

In [5]:
class Car:
    def __init__(self, env, name, car_tank_size, car_tank_level, gas_station, logger):
        '''
        Initialize the car with the following parameters:
        - env: the simpy environment
        - name: the name of the car
        - car_tank_size: the size of the car's fuel tank
        - car_tank_level: the range of fuel levels the car can arrive with
        - gas_station: the gas station object
        - logger: the logger object to log events
        '''

        self.env = env
        self.name = name
        self.car_tank_size = car_tank_size
        self.car_tank_level = random.randint(car_tank_level[0], car_tank_level[1])

        self.logger = logger

        self.env.process(self.run(gas_station))

    def run(self, gas_station):
        '''
        The flows of the car through the gas station
        - gas_station: the gas station object
        '''

        print(f'{self.env.now:6.1f} s: {self.name} arrived at gas station')
        self.logger.log_car_arrival(self.name, self.env.now, gas_station.fuel_tank.level)

        with gas_station.fuel_pump.request() as req:
            # Request one of the gas pumps
            yield req

            # Get the required amount of fuel
            fuel_required = self.car_tank_size - self.car_tank_level
            
            print(f'{self.env.now:6.1f} s: {self.name} requires {fuel_required:.1f}L to refuel')
            self.logger.log_car_fuel_request(self.name, self.env.now, fuel_required, gas_station.fuel_tank.level)

            yield self.env.process(gas_station.refuel_car(fuel_required))

            print(f'{self.env.now:6.1f} s: {self.name} refueled with {fuel_required:.1f}L')
            self.logger.log_car_departure(self.name, self.env.now, gas_station.fuel_tank.level)

### Entity Generation

In [6]:
def car_generator(env, t_inter, gas_station, car_tank_size, car_tank_level, logger):
    '''
    The process that generates cars arriving at the gas station
    - t_inter: the interval between car arrivals
    - gas_station: the gas station object
    - car_tank_size: the size of the car's fuel tank
    - car_tank_level: the range of fuel levels the car can arrive with
    - logger: the logger object to log events
    '''

    i = 0
    while True:
        yield env.timeout(random.randint(t_inter[0], t_inter[1])) #equivalently to random.randint(*t_inter)
        Car(env, f'Car {i}', car_tank_size, car_tank_level, gas_station, logger)
        i += 1

### Run Simulation

Unlike the previous examples, now we have two processes defined "outside" (this is a design decision): the car generator process and the tank control process.

In [7]:
RANDOM_SEED = 42           # Random seed
SIM_TIME = 1000            # Simulation time (seconds)     

#resource params
STATION_TANK_SIZE = 200    # Size of the gas station tank (liters)
THRESHOLD = 25             # Station tank minimum level (% of full)
REFUELING_SPEED = 2        # Rate of refuelling car fuel tank (liters / second)
TANK_TRUCK_TIME = 300      # Time it takes tank truck to arrive (seconds)
N_FUEL_PUMPS = 1           # Number of fuel pumps

#car (entity) params
CAR_TANK_SIZE = 50         # Size of car fuel tanks (liters)
CAR_TANK_LEVEL = [5, 25]   # Min/max levels of car fuel tanks (liters)

#entity generation params
T_INTER = [30, 300]        # Interval between car arrivals [min, max] (seconds)

# Setup and start the simulation
print('Gas Station refuelling')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env = simpy.Environment()

#define logger
logger = EventLogger()

#define resources
gas_station = GasStation(env, N_FUEL_PUMPS, STATION_TANK_SIZE, REFUELING_SPEED, TANK_TRUCK_TIME, logger)

#define processes
env.process(car_generator(env, T_INTER, gas_station, CAR_TANK_SIZE, CAR_TANK_LEVEL, logger))
env.process(gas_station.fuel_tank_control(THRESHOLD))

#Execute
env.run(until=SIM_TIME)
print('... Done \n')

Gas Station refuelling
Running Simulation...
  87.0 s: Car 0 arrived at gas station
  87.0 s: Car 0 requires 45.0L to refuel
 109.5 s: Car 0 refueled with 45.0L
 257.0 s: Car 1 arrived at gas station
 257.0 s: Car 1 requires 38.0L to refuel
 276.0 s: Car 1 refueled with 38.0L
 401.0 s: Car 2 arrived at gas station
 401.0 s: Car 2 requires 41.0L to refuel
 421.5 s: Car 2 refueled with 41.0L
 483.0 s: Car 3 arrived at gas station
 483.0 s: Car 3 requires 28.0L to refuel
 490.0 s: Calling tank truck
 497.0 s: Car 3 refueled with 28.0L
 557.0 s: Car 4 arrived at gas station
 557.0 s: Car 4 requires 27.0L to refuel
 570.5 s: Car 4 refueled with 27.0L
 790.0 s: Tank truck arrived and refuelled station with 179.0L
 803.0 s: Car 5 arrived at gas station
 803.0 s: Car 5 requires 44.0L to refuel
 825.0 s: Car 5 refueled with 44.0L
 848.0 s: Car 6 arrived at gas station
 848.0 s: Car 6 requires 43.0L to refuel
 869.5 s: Car 6 refueled with 43.0L
 989.0 s: Car 7 arrived at gas station
 989.0 s: Ca

And thanks to the `EventLogger`, we also have a DataFrame (table) that reflects the same information: 

In [8]:
events_df = logger.get_logs_df()

events_df

,event_time,event_name,event_key,fuel_tank_level,fuel_required
0,87.0,entity_arrival,Car 0,200,NaN
1,87.0,entity_fuel_request,Car 0,200,45.0
2,109.5,entity_departure,Car 0,155,NaN
3,257.0,entity_arrival,Car 1,155,NaN
4,257.0,entity_fuel_request,Car 1,155,38.0
5,276.0,entity_departure,Car 1,117,NaN
6,401.0,entity_arrival,Car 2,117,NaN
7,401.0,entity_fuel_request,Car 2,117,41.0
8,421.5,entity_departure,Car 2,76,NaN
9,483.0,entity_arrival,Car 3,76,NaN


# Exercise / Tasks

**Tasks are independently of each other.**

**Note: Classes / Functions are named differently for each task to have always the "original" version without changes.**

## Task 1

We'd like to model vehicles with different tank sizes (`CAR_TANK_SIZE`). For doing so:

-----

**Task 1.1: Replace `CAR_TANK_SIZE` for a list (range) `[50, 100]`. Modify the `car_tank_size` attribute to pick one random value between the defined range.**

**Task 1.2: Modify the `car_tank_level` attribute to be a random value between 5 and the tank size. Note that `car_tank_level` is not anymore an "external" parameter (i.e. `CAR_TANK_LEVEL` is not used)**

**Task 1.3: Modify the `EventLogger_Task1` class to log the tank size for every car. You must create a new column (e.g. `car_tank_size`) to store this information to the DataFrame.**

In [ ]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class EventLogger_Task1: 
    def __init__(self):
        self.logs = []
    
    def log_car_arrival(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_arrival', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})
    
    def log_car_fuel_request(self, entity, time, fuel_required, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_fuel_request', 'event_key': entity, 'fuel_required': fuel_required, 'fuel_tank_level': fuel_tank_level})

    def log_car_departure(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_departure', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})

    def log_truck_call(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_call', 'fuel_tank_level': fuel_tank_level})
    
    def log_truck_arrival(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_arrival', 'fuel_tank_level': fuel_tank_level})

    def log_truck_departure(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_departure', 'fuel_tank_level': fuel_tank_level})

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [165]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class GasStation_Task1: 
    def __init__(self, env, n_fuel_pumps, station_tank_size, refueling_speed, tank_truck_time, logger):

        self.env = env
        self.fuel_pump = simpy.Resource(self.env, capacity=n_fuel_pumps)
        self.fuel_tank = simpy.Container(self.env, capacity=station_tank_size, init=station_tank_size)

        self.refueling_speed = refueling_speed
        self.tank_truck_time = tank_truck_time
        self.truck_called = False
        
        self.logger = logger


    def refuel_car(self, fuel_required):

        yield self.fuel_tank.get(fuel_required)
        yield self.env.timeout(fuel_required / self.refueling_speed)

    def refill_fuel_tank(self):
        self.truck_called = True
        
        yield self.env.timeout(self.tank_truck_time)
        self.logger.log_truck_arrival(self.env.now, self.fuel_tank.level)

        amount = self.fuel_tank.capacity - self.fuel_tank.level
        yield self.fuel_tank.put(amount)
        
        print(f'{self.env.now:6.1f} s: Tank truck arrived and refuelled station with {amount:.1f}L')
        self.logger.log_truck_departure(self.env.now, self.fuel_tank.level)
        self.truck_called = False

    def fuel_tank_control(self, threshold):
        while True:
            fuel_tank_level_ratio = 100 * (self.fuel_tank.level / self.fuel_tank.capacity)
            if (fuel_tank_level_ratio < threshold) and (not self.truck_called):
                
                print(f'{self.env.now:6.1f} s: Calling tank truck')
                self.logger.log_truck_call(self.env.now, self.fuel_tank.level)
                
                
                yield self.env.process(self.refill_fuel_tank())

            yield self.env.timeout(10)



In [51]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class Car_Task1: 
    def __init__(self, env, name, car_tank_size, car_tank_level, gas_station, logger):
        self.env = env
        self.name = name
        self.car_tank_size = car_tank_size
        self.car_tank_level = random.randint(car_tank_level[0], car_tank_level[1])

        self.logger = logger

        self.env.process(self.run(gas_station))

    def run(self, gas_station):
  
        print(f'{self.env.now:6.1f} s: {self.name} arrived at gas station')
        self.logger.log_car_arrival(self.name, self.env.now, gas_station.fuel_tank.level)

        with gas_station.fuel_pump.request() as req:
            yield req

            fuel_required = self.car_tank_size - self.car_tank_level
            
            print(f'{self.env.now:6.1f} s: {self.name} requires {fuel_required:.1f}L to refuel')
            self.logger.log_car_fuel_request(self.name, self.env.now, fuel_required, gas_station.fuel_tank.level)

            yield self.env.process(gas_station.refuel_car(fuel_required))

            print(f'{self.env.now:6.1f} s: {self.name} refueled with {fuel_required:.1f}L')
            self.logger.log_car_departure(self.name, self.env.now, gas_station.fuel_tank.level)

In [52]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE FUNCTION NAME
def car_generator_task1(env, t_inter, gas_station, car_tank_size, car_tank_level, logger): 
    i = 0
    while True:
        yield env.timeout(random.randint(t_inter[0], t_inter[1]))
        Car(env, f'Car {i}', car_tank_size, car_tank_level, gas_station, logger)
        i += 1

In [ ]:
#LEAVE THIS CODE AS IT IS

RANDOM_SEED = 42           # Random seed
SIM_TIME = 1000            # Simulation time (seconds)     

#resource params
STATION_TANK_SIZE = 200    # Size of the gas station tank (liters)
THRESHOLD = 25             # Station tank minimum level (% of full)
REFUELING_SPEED = 2        # Rate of refuelling car fuel tank (liters / second)
TANK_TRUCK_TIME = 300      # Time it takes tank truck to arrive (seconds)
N_FUEL_PUMPS = 1           # Number of fuel pumps

#car (entity) params
CAR_TANK_SIZE = [50, 100]   # Size of car fuel tanks (liters)

#entity generation params
T_INTER = [30, 300]        # Interval between car arrivals [min, max] (seconds)

# Setup and start the simulation
print('Gas Station refuelling')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env_t1 = simpy.Environment()

#define logger
logger_t1 = EventLogger_Task1()

#define resources
gas_station_t1 = GasStation_Task1(env_t1, N_FUEL_PUMPS, STATION_TANK_SIZE, REFUELING_SPEED, TANK_TRUCK_TIME, logger_t1)

#define processes
env_t1.process(car_generator_task1(env_t1, T_INTER, gas_station_t1, CAR_TANK_SIZE, logger_t1))
env_t1.process(gas_station_t1.fuel_tank_control(THRESHOLD))

#Execute
env_t1.run(until=SIM_TIME)
print('... Done \n')

In [ ]:
#LEAVE THIS CODE AS IT IS
events_df_t1 = logger_t1.get_logs_df()

events_df_t1

Answer the following questions:

1. Under this new scenario, at what time is the truck called? is it before or after compared to the original version?
1. Under this new scenario, how many liters are in the tank at the truck departure time? If is not the same as the tank capacity (i.e. `STATION_TANK_SIZE`), why?

**Hint:** Compare `events_df_t1` with `events_df` to answer.

**Answer in this markdown chunk:**

1. *Your answer here* 
1. *Your answer here*

## Task 2

We'd like to model different types of fuel (petrol and diesel) each with its own container. The fuel pumps are able to handle both types of fuel indistinctly. We'll assume also that each time the truck is called, both containers are refilled. For doing so:

-----

**Task 2.1: Add an attribute to the `Car_Task2` to store the type of vehicle (`petrol` or `diesel`). The type of vehicle must be randomly selected from a uniform distribution.**

**Task 2.2: Add another resource in the `GasStation_Task2` to store the diesel container and modify the `refuel_car` to get the corresponding fuel depending on the type of vehicle.**

**Task 2.3: Modify the tank control process, to call the truck if the petrol *or* diesel (either of the two) tank ratio is below the threshold.** 

**Task 2.4: Modify the process for refilling the tanks, to fill both containers to capacity.** 

**Task 2.5: Modify the `EventLogger_Task2` class to log the type of the fuel in another column (`petrol` or `diesel`). Note that in this new scenario, the `fuel_tank_level` must be from the corresponding fuel.** 

-----

**Hint: For Task 2.5, don't overthink it. You can add two different rows (one for `petrol` and another for `diesel`) in the events that involves the two types of fuels (such as the events related to the truck (call, arrival and departure))**

In [169]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class EventLogger_Task2: 
    def __init__(self):
        self.logs = []
    
    def log_car_arrival(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_arrival', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})
    
    def log_car_fuel_request(self, entity, time, fuel_required, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_fuel_request', 'event_key': entity, 'fuel_required': fuel_required, 'fuel_tank_level': fuel_tank_level})

    def log_car_departure(self, entity, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'entity_departure', 'event_key': entity, 'fuel_tank_level': fuel_tank_level})

    def log_truck_call(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_call', 'fuel_tank_level': fuel_tank_level})
    
    def log_truck_arrival(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_arrival', 'fuel_tank_level': fuel_tank_level})

    def log_truck_departure(self, time, fuel_tank_level):
        self.logs.append({'event_time': time, 'event_name': 'truck_departure', 'fuel_tank_level': fuel_tank_level})

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [170]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class GasStation_Task2: 
    def __init__(self, env, n_fuel_pumps, station_tank_size, refueling_speed, tank_truck_time, logger):

        self.env = env
        self.fuel_pump = simpy.Resource(self.env, capacity=n_fuel_pumps)
        self.fuel_tank = simpy.Container(self.env, capacity=station_tank_size, init=station_tank_size)

        self.refueling_speed = refueling_speed
        self.tank_truck_time = tank_truck_time
        self.truck_called = False
        
        self.logger = logger


    def refuel_car(self, fuel_required):

        yield self.fuel_tank.get(fuel_required)
        yield self.env.timeout(fuel_required / self.refueling_speed)

    def refill_fuel_tank(self):
        self.truck_called = True
        
        yield self.env.timeout(self.tank_truck_time)
        self.logger.log_truck_arrival(self.env.now, self.fuel_tank.level)

        amount = self.fuel_tank.capacity - self.fuel_tank.level
        yield self.fuel_tank.put(amount)
        
        print(f'{self.env.now:6.1f} s: Tank truck arrived and refuelled station with {amount:.1f}L')
        self.logger.log_truck_departure(self.env.now, self.fuel_tank.level)
        self.truck_called = False

    def fuel_tank_control(self, threshold):
        while True:
            fuel_tank_level_ratio = 100 * (self.fuel_tank.level / self.fuel_tank.capacity)
            if (fuel_tank_level_ratio < threshold) and (not self.truck_called):
                
                print(f'{self.env.now:6.1f} s: Calling tank truck')
                self.logger.log_truck_call(self.env.now, self.fuel_tank.level)
                
                
                yield self.env.process(self.refill_fuel_tank())

            yield self.env.timeout(10)



In [171]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE CLASS NAME
class Car_Task2: 
    def __init__(self, env, name, car_tank_size, car_tank_level, gas_station, logger):
        self.env = env
        self.name = name
        self.car_tank_size = car_tank_size
        self.car_tank_level = random.randint(car_tank_level[0], car_tank_level[1])

        self.logger = logger

        self.env.process(self.run(gas_station))

    def run(self, gas_station):
  
        print(f'{self.env.now:6.1f} s: {self.name} arrived at gas station')
        self.logger.log_car_arrival(self.name, self.env.now, gas_station.fuel_tank.level)

        with gas_station.fuel_pump.request() as req:
            yield req

            fuel_required = self.car_tank_size - self.car_tank_level
            
            print(f'{self.env.now:6.1f} s: {self.name} requires {fuel_required:.1f}L to refuel')
            self.logger.log_car_fuel_request(self.name, self.env.now, fuel_required, gas_station.fuel_tank.level)

            yield self.env.process(gas_station.refuel_car(fuel_required))

            print(f'{self.env.now:6.1f} s: {self.name} refueled with {fuel_required:.1f}L')
            self.logger.log_car_departure(self.name, self.env.now, gas_station.fuel_tank.level)

In [59]:
#IMPLEMENT YOUR CHANGES (IF NECCESARY) HERE. DO NOT CHANGE THE FUNCTION NAME
def car_generator_task2(env, t_inter, gas_station, car_tank_size, car_tank_level, logger): 
    i = 0
    while True:
        yield env.timeout(random.randint(t_inter[0], t_inter[1]))
        Car(env, f'Car {i}', car_tank_size, car_tank_level, gas_station, logger)
        i += 1

In [ ]:
#LEAVE THIS CODE AS IT IS

RANDOM_SEED = 42           # Random seed
SIM_TIME = 1500            # Simulation time (seconds)     

#resource params
STATION_TANK_SIZE = 200    # Size of the gas station tank (liters)
THRESHOLD = 25             # Station tank minimum level (% of full)
REFUELING_SPEED = 2        # Rate of refuelling car fuel tank (liters / second)
TANK_TRUCK_TIME = 300      # Time it takes tank truck to arrive (seconds)
N_FUEL_PUMPS = 1           # Number of fuel pumps

#car (entity) params
CAR_TANK_SIZE = 50         # Size of car fuel tanks (liters)
CAR_TANK_LEVEL = [5, 25]   # Min/max levels of car fuel tanks (liters)

#entity generation params
T_INTER = [30, 300]        # Interval between car arrivals [min, max] (seconds)

# Setup and start the simulation
print('Gas Station refuelling')
print('Running Simulation...')
random.seed(RANDOM_SEED)
env_t2 = simpy.Environment()

#define logger
logger_t2 = EventLogger_Task2()

#define resources
gas_station_t2 = GasStation_Task2(env_t2, N_FUEL_PUMPS, STATION_TANK_SIZE, REFUELING_SPEED, TANK_TRUCK_TIME, logger_t2)

#define processes
env_t2.process(car_generator_task2(env_t2, T_INTER, gas_station_t2, CAR_TANK_SIZE, CAR_TANK_LEVEL, logger_t2))
env_t2.process(gas_station_t2.tank_control(THRESHOLD))

#Execute
env_t2.run(until=SIM_TIME)
print('... Done \n')

In [ ]:
#LEAVE THIS CODE AS IT IS
events_df_t2 = logger_t2.get_logs_df()

events_df_t2

Answer the following questions:

- If you have not touched the parameters, you should have only 1 truck call. At what time and due to what type of fuel was the truck called?

**Answer in this markdown chunk:**

-  *Your answer here*